In [4]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
# --- Standard Libraries ---
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
import numpy as np
import gc

# --- Machine Learning Libraries ---
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from lightgbm import LGBMRegressor # Our main model for CQR
import xgboost as xgb
import optuna

print("Libraries imported successfully.")

# --- Helper Function for Winkler Score ---
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)

# --- Global Constants ---
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
COMPETITION_ALPHA = 0.1 # This is the alpha for the competition metric (90% interval)

# --- Load Raw Data ---
try:
    drop_cols=['id', 'golf', 'view_rainier', 'view_skyline', 'view_lakesamm', 'view_otherwater', 'view_other']
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv').drop(columns=drop_cols)
    df_test = pd.read_csv(DATA_PATH + 'test.csv').drop(columns=drop_cols)
    print("Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()

# --- Prepare Target Variable ---
y_true = df_train['sale_price'].copy()
print("Setup complete.")

Libraries imported successfully.
Raw data loaded successfully.
Setup complete.


In [5]:
# =============================================================================
# BLOCK 2: SYNTHESIZED FEATURE ENGINEERING
# =============================================================================
print("--- Starting Block 2: Synthesized Feature Engineering ---")

def create_synthesized_features(df_train, df_test):
    # Combine for consistent processing and reset the index
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    train_ids = df_train.index
    test_ids = df_test.index
    all_data = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)

    # A) Brute-Force Numerical Interactions
    NUMS = ['area', 'land_val', 'imp_val', 'sqft_lot', 'sqft', 'sqft_1', 'grade', 'year_built']
    for i in range(len(NUMS)):
        for j in range(i + 1, len(NUMS)):
            all_data[f'{NUMS[i]}_x_{NUMS[j]}'] = all_data[NUMS[i]] * all_data[NUMS[j]]

    # B) Date Features
    all_data['sale_date'] = pd.to_datetime(all_data['sale_date'])
    all_data['year'] = all_data['sale_date'].dt.year
    all_data['month'] = all_data['sale_date'].dt.month
    all_data['year_diff'] = all_data['year'] - all_data['year_built']
    
    # C) TF-IDF Text Features
    text_cols = ['subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data[text_cols] = all_data[text_cols].fillna('missing').astype(str)
    for col in text_cols:
        tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), max_features=128, binary=True)
        tfidf_matrix = tfidf.fit_transform(all_data[col])
        svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
        tfidf_svd = svd.fit_transform(tfidf_matrix)
        tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
        all_data = pd.concat([all_data, tfidf_df], axis=1)

    # D) Log transform
    for c in ['land_val_x_imp_val', 'land_val_x_sqft', 'imp_val_x_sqft']:
        if c in all_data.columns:
            all_data[c] = np.log1p(all_data[c].fillna(0))

    # E) Final Cleanup
    cols_to_drop = ['sale_date', 'subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data = all_data.drop(columns=cols_to_drop)
    all_data.fillna(0, inplace=True)

    # Separate final datasets
    X = all_data[all_data['is_train'] == 1].drop(columns=['is_train', 'sale_price'])
    X_test = all_data[all_data['is_train'] == 0].drop(columns=['is_train', 'sale_price'])
    
    return X, X_test, test_ids # Return test_ids for the final submission

# --- Run Feature Engineering ---
X, X_test, test_ids = create_synthesized_features(df_train, df_test)
print(f"\nSynthesized FE complete. Total features: {X.shape[1]}")
gc.collect()

--- Starting Block 2: Synthesized Feature Engineering ---

Synthesized FE complete. Total features: 111


988

In [ ]:
# =============================================================================
# BLOCK 3: CQR WITH XGBOOST & K-FOLD AWARE OPTUNA TUNING (CORRECTED)
# =============================================================================
print("\n--- Starting XGBoost CQR Pipeline with Hyperparameter Tuning ---")

# --- Initialize K-Fold and Data ---
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
grade_for_stratify = pd.read_csv(DATA_PATH + 'dataset.csv')['grade']
LOWER_ALPHA = 0.05
UPPER_ALPHA = 0.95

# ==============================================================================
# PART 1: K-FOLD AWARE TUNING FOR XGBOOST QUANTILE MODELS
# ==============================================================================
print("\n# Tuning XGBoost Quantile Models with K-Fold Aware Optuna...")

def objective_xgb_quantile_kfold(trial):
    params = {
        'objective': 'reg:quantileerror',
        'eval_metric': 'rmse',
        'tree_method': 'hist', 'random_state': RANDOM_STATE, 'n_jobs': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2500),
        'max_depth': trial.suggest_int('max_depth', 6, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.95),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 0.1, 10.0, log=True),
    }
    
    oof_winkler_scores = []
    
    for train_idx, val_idx in skf.split(X, grade_for_stratify):
        X_train_fold, X_calib_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_calib_fold = y_true.iloc[train_idx], y_true.iloc[val_idx]

        # ==================== THE FIX IS HERE ====================
        # Move early_stopping_rounds into the constructor for both models
        
        # Train lower model
        model_lower = xgb.XGBRegressor(**params, quantile_alpha=LOWER_ALPHA, early_stopping_rounds=50)
        model_lower.fit(X_train_fold, y_train_fold, eval_set=[(X_calib_fold, y_calib_fold)], verbose=False)

        # Train upper model
        model_upper = xgb.XGBRegressor(**params, quantile_alpha=UPPER_ALPHA, early_stopping_rounds=50)
        model_upper.fit(X_train_fold, y_train_fold, eval_set=[(X_calib_fold, y_calib_fold)], verbose=False)
        # =========================================================

        calib_pred_lower = model_lower.predict(X_calib_fold)
        calib_pred_upper = model_upper.predict(X_calib_fold)
        
        nonconformity_scores = np.maximum(calib_pred_lower - y_calib_fold, y_calib_fold - calib_pred_upper)
        q_correction = np.quantile(nonconformity_scores, (1 - COMPETITION_ALPHA) * (1 + 1/len(y_calib_fold)))
        
        fold_lower = calib_pred_lower - q_correction
        fold_upper = calib_pred_upper + q_correction
        fold_score, _ = winkler_score(y_calib_fold, fold_lower, fold_upper, return_coverage=True)
        oof_winkler_scores.append(fold_score)
        
    return np.mean(oof_winkler_scores)

# --- Run the Optuna Study ---
N_OPTUNA_TRIALS_CQR = 20
study_quantile_xgb = optuna.create_study(direction='minimize')
study_quantile_xgb.optimize(objective_xgb_quantile_kfold, n_trials=N_OPTUNA_TRIALS_CQR)

best_params_quantile = study_quantile_xgb.best_params
print(f"\n# XGBoost Quantile Model Tuning Complete. Best Avg Winkler Score: ${study_quantile_xgb.best_value:,.2f}")

# ==============================================================================
# PART 2: FINAL XGBOOST CQR TRAINING WITH TUNED PARAMETERS
# ==============================================================================
print("\n# Final XGBoost CQR Training with Tuned Hyperparameters...")

oof_lower = np.zeros(len(X))
oof_upper = np.zeros(len(X))
final_test_lower = np.zeros(len(X_test))
final_test_upper = np.zeros(len(X_test))

final_params_quantile = {
    'objective': 'reg:quantileerror', 'eval_metric': 'rmse', 'tree_method': 'hist',
    'random_state': RANDOM_STATE, 'n_jobs': -1,
    **best_params_quantile
}

for fold, (train_idx, val_idx) in enumerate(skf.split(X, grade_for_stratify)):
    print(f"\n# FOLD {fold+1}/{N_SPLITS}")
    X_train_fold, X_calib_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_calib_fold = y_true.iloc[train_idx], y_true.iloc[val_idx]

    # ==================== APPLYING THE SAME FIX HERE ====================
    model_lower = xgb.XGBRegressor(**final_params_quantile, quantile_alpha=LOWER_ALPHA, early_stopping_rounds=50)
    model_lower.fit(X_train_fold, y_train_fold, eval_set=[(X_calib_fold, y_calib_fold)], verbose=False)

    model_upper = xgb.XGBRegressor(**final_params_quantile, quantile_alpha=UPPER_ALPHA, early_stopping_rounds=50)
    model_upper.fit(X_train_fold, y_train_fold, eval_set=[(X_calib_fold, y_calib_fold)], verbose=False)
    # ====================================================================
    
    calib_pred_lower = model_lower.predict(X_calib_fold)
    calib_pred_upper = model_upper.predict(X_calib_fold)
    
    nonconformity_scores = np.maximum(calib_pred_lower - y_calib_fold, y_calib_fold - calib_pred_upper)
    q_correction = np.quantile(nonconformity_scores, (1 - COMPETITION_ALPHA) * (1 + 1/len(y_calib_fold)))
    print(f"  Correction value 'q' for this fold: {q_correction:.2f}")

    oof_lower[val_idx] = calib_pred_lower - q_correction
    oof_upper[val_idx] = calib_pred_upper + q_correction
    final_test_lower += (model_lower.predict(X_test) - q_correction) / N_SPLITS
    final_test_upper += (model_upper.predict(X_test) + q_correction) / N_SPLITS

# --- Final Evaluation ---
print("\n" + "="*50)
print("# XGBoost CQR K-Fold Training Complete.")
final_winkler_score, final_coverage = winkler_score(y_true, oof_lower, oof_upper, return_coverage=True)
print(f"# Final OOF Winkler Score: {final_winkler_score:,.2f}")
print(f"# Final OOF Coverage: {final_coverage:.2%}")
print("="*50)


--- Starting XGBoost CQR Pipeline with Hyperparameter Tuning ---


[I 2025-07-09 15:38:55,991] A new study created in memory with name: no-name-d7a146f0-25fe-41e5-9401-b5ec702057f6



# Tuning XGBoost Quantile Models with K-Fold Aware Optuna...


[I 2025-07-09 15:43:45,106] Trial 0 finished with value: 340849.17919288424 and parameters: {'learning_rate': 0.04418648971043228, 'n_estimators': 1465, 'max_depth': 6, 'subsample': 0.7641867339772052, 'colsample_bytree': 0.8417870915469254, 'min_child_weight': 3, 'lambda': 4.488571315979643, 'alpha': 0.7239464065364105}. Best is trial 0 with value: 340849.17919288424.


In [ ]:
# =============================================================================
# BLOCK 4: FINAL SUBMISSION
# =============================================================================
print("\n--- STAGE 3: Creating Final Submission File ---")

# Ensure the upper bound is always greater than or equal to the lower bound
final_test_upper = np.maximum(final_test_lower, final_test_upper)

# Create submission dataframe
submission_df = pd.DataFrame({
    'id': test_ids,
    'pi_lower': final_test_lower,
    'pi_upper': final_test_upper
})

submission_df.to_csv('submission_cqr_v1.csv', index=False)
print("\n'submission_cqr_v1.csv' created successfully!")
display(submission_df.head())